In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove*.zip

In [ ]:
! pip install glove-python-binary

In [ ]:
from glove import Corpus, Glove
import pandas as pd
import sys
import os
from tqdm import trange
import re
import csv
import numpy as np

In [ ]:
def preprocess(text):
  processed_text = text.replace('\\', '')
  symbols_regex = re.compile(r"([.()[\]{}!?$@&#*/_;,`~:\-<>\+=])")
  processed_text = symbols_regex.sub(" \\1 ", processed_text)
  processed_text = re.sub(r'\s+', ' ', processed_text)
  processed_text = processed_text.replace('[ CITE ]', '[CITE]')
  processed_text = processed_text.replace('"', ' " ')
  processed_text = processed_text.replace("'", " ' ")
  processed_text = processed_text.strip()
  # if processed_text.isupper():
  processed_text = processed_text.lower()
  return re.sub(r'\s+', ' ', processed_text)

In [ ]:
case_sentence_csv_folder = '/content/drive/Shareddrives/SigmaLaw-WPP/criminal_sentence_dataset/'
csv_file_list = [
  'sentence_dataset_1000_cases.csv'
  , 'sentence_dataset_2000_cases.csv', 'sentence_dataset_3000_cases.csv', 
  'sentence_dataset_4000_cases.csv', 'sentence_dataset_5000_cases.csv', 'sentence_dataset_6000_cases.csv', 
  'sentence_dataset_7000_cases.csv', 'sentence_dataset_8000_cases.csv', 'sentence_dataset_9000_cases.csv',
  'sentence_dataset_10000_cases.csv'
]

In [ ]:
train_samples = []

for findex in trange(len(csv_file_list)):
  df = pd.read_csv(os.path.join(case_sentence_csv_folder, csv_file_list[findex]))
  for index, row in df.iterrows():
    sentence = row['sentence'].strip().lower()
    train_samples.append(preprocess(sentence).split(' '))

100%|██████████| 10/10 [03:58<00:00, 23.83s/it]


In [ ]:
train_samples[0]

['soon',
 'thereafter',
 ',',
 'acosta',
 'entered',
 'an',
 'abandoned',
 'factory',
 'building',
 'at',
 'north',
 '10th',
 'street',
 'and',
 'kent',
 'avenue',
 'and',
 ',',
 'armed',
 'with',
 'a',
 'knife',
 ',',
 'demanded',
 'money',
 'from',
 'debra',
 'perry',
 ',',
 'another',
 'prostitute',
 'who',
 'lived',
 'in',
 'the',
 'building',
 ',',
 'and',
 'dennis',
 'cetter',
 ',',
 'the',
 'man',
 'with',
 'whom',
 'perry',
 'had',
 'spent',
 'part',
 'of',
 'the',
 'day',
 '.']

In [ ]:
def glove2dict(glove_filename):
    with open(glove_filename, encoding='utf-8') as f:
        reader = csv.reader(f, delimiter=' ', quoting=csv.QUOTE_NONE)
        embed = {line[0]: np.array(list(map(float, line[1:])))
                for line in reader}
    return embed



In [ ]:
glove_path = "glove.6B.50d.txt" # get it from https://nlp.stanford.edu/projects/glove
pre_glove = glove2dict(glove_path)

In [ ]:
a = pre_glove.keys()

In [ ]:
all_words = []
for i in a:
  all_words.append(i)

In [ ]:
train_samples.append(all_words)

In [ ]:
corpus = Corpus() 

#Training the corpus to generate the co-occurrence matrix which is used in GloVe
corpus.fit(train_samples, ignore_missing=True)

In [ ]:
glove = Glove(no_components=300, learning_rate=0.05) 
glove.fit(corpus.matrix, epochs=100, no_threads=4, verbose=True)

Performing 100 training epochs with 4 threads
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24
Epoch 25
Epoch 26
Epoch 27
Epoch 28
Epoch 29
Epoch 30
Epoch 31
Epoch 32
Epoch 33
Epoch 34
Epoch 35
Epoch 36
Epoch 37
Epoch 38
Epoch 39
Epoch 40
Epoch 41
Epoch 42
Epoch 43
Epoch 44
Epoch 45
Epoch 46
Epoch 47
Epoch 48
Epoch 49
Epoch 50
Epoch 51
Epoch 52
Epoch 53
Epoch 54
Epoch 55
Epoch 56
Epoch 57
Epoch 58
Epoch 59
Epoch 60
Epoch 61
Epoch 62
Epoch 63
Epoch 64
Epoch 65
Epoch 66
Epoch 67
Epoch 68
Epoch 69
Epoch 70
Epoch 71
Epoch 72
Epoch 73
Epoch 74
Epoch 75
Epoch 76
Epoch 77
Epoch 78
Epoch 79
Epoch 80
Epoch 81
Epoch 82
Epoch 83
Epoch 84
Epoch 85
Epoch 86
Epoch 87
Epoch 88
Epoch 89
Epoch 90
Epoch 91
Epoch 92
Epoch 93
Epoch 94
Epoch 95
Epoch 96
Epoch 97
Epoch 98
Epoch 99


In [ ]:
glove.add_dictionary(corpus.dictionary)

In [ ]:
len(glove.word_vectors[0])

300

In [ ]:
glove.most_similar('petitioner')

[('habeas', 0.5400699739640803),
 ('plaintiff', 0.5125564289972089),
 ('appellant', 0.5086415172796974),
 ('demonstrate', 0.49417870767233324)]

In [ ]:
glove.most_similar('justice')

[('obstruction', 0.5517622394281759),
 ('scalia', 0.5470743582362398),
 ('kennedy', 0.5115840173811296),
 ('souter', 0.49635314423101523)]

In [ ]:
new_glove.most_similar('defendant',number=11)

[('petitioner', 0.4128596334215855),
 ('asserts', 0.3154408141152707),
 ('contends', 0.3035594375724916),
 ('however', 0.29400557034296065),
 ('finally', 0.2938967849394781),
 ('argued', 0.2927174928369545),
 ('htm', 0.2885545834088179),
 ('moreover', 0.2875291537687086),
 ('said', 0.2852676035347912),
 ('complains', 0.2832472979251031)]

In [ ]:
new_glove = Glove.load('/content/drive/Shareddrives/SigmaLaw-WPP/EAMS/glove/glove_100_epochs_300d/glove_100_epochs_300d.model')

In [ ]:
len(new_glove.dictionary.keys())

84606

In [ ]:
saved_model_path = "/content/glove_100_epochs_300d_updated/"
!mkdir {saved_model_path}

In [ ]:
glove.save('/content/glove_100_epochs_300d_updated/glove_100_epochs_300d_updated.model')

In [ ]:
!du -sh {saved_model_path}

391M	/content/glove_100_epochs_300d_updated/


In [ ]:
!cp -av {saved_model_path} /content/drive/Shareddrives/SigmaLaw-WPP/EAMS/glove

'/content/glove_100_epochs_300d_updated/' -> '/content/drive/Shareddrives/SigmaLaw-WPP/EAMS/glove/glove_100_epochs_300d_updated'
'/content/glove_100_epochs_300d_updated/glove_100_epochs_300d_updated.model' -> '/content/drive/Shareddrives/SigmaLaw-WPP/EAMS/glove/glove_100_epochs_300d_updated/glove_100_epochs_300d_updated.model'


In [ ]:
glove.dictionary

{'soon': 0,
 'thereafter': 1,
 ',': 2,
 'acosta': 3,
 'entered': 4,
 'an': 5,
 'abandoned': 6,
 'factory': 7,
 'building': 8,
 'at': 9,
 'north': 10,
 '10th': 11,
 'street': 12,
 'and': 13,
 'kent': 14,
 'avenue': 15,
 'armed': 16,
 'with': 17,
 'a': 18,
 'knife': 19,
 'demanded': 20,
 'money': 21,
 'from': 22,
 'debra': 23,
 'perry': 24,
 'another': 25,
 'prostitute': 26,
 'who': 27,
 'lived': 28,
 'in': 29,
 'the': 30,
 'dennis': 31,
 'cetter': 32,
 'man': 33,
 'whom': 34,
 'had': 35,
 'spent': 36,
 'part': 37,
 'of': 38,
 'day': 39,
 '.': 40,
 'testified': 41,
 'that': 42,
 'she': 43,
 'knew': 44,
 'neighborhood': 45,
 'having': 46,
 'seen': 47,
 'him': 48,
 'almost': 49,
 'daily': 50,
 'for': 51,
 'last': 52,
 'month': 53,
 'used': 54,
 'drugs': 55,
 'as': 56,
 'searched': 57,
 'through': 58,
 'couple': 59,
 "'": 60,
 's': 61,
 'clothing': 62,
 'looking': 63,
 'reached': 64,
 'baseball': 65,
 'bat': 66,
 'but': 67,
 'took': 68,
 'it': 69,
 'her': 70,
 'to': 71,
 'hit': 72,
 'fight'

In [ ]:
!pip install mittens

In [ ]:

import csv
import numpy as np
from collections import Counter
from nltk.corpus import brown
from mittens import GloVe, Mittens
from sklearn.feature_extraction import _stop_words
from sklearn.feature_extraction.text import CountVectorizer
import nltk

In [ ]:
nltk.download('brown')

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.


True

In [ ]:


sw = list(_stop_words.ENGLISH_STOP_WORDS)
brown_data = brown.words()[:200000]
brown_nonstop = [token.lower() for token in brown_data if (token.lower() not in sw)]
oov = [token for token in brown_nonstop if token not in pre_glove.keys()]

def get_rareoov(xdict, val):
    return [k for (k,v) in Counter(xdict).items() if v<=val]

#oov_rare = get_rareoov(oov, 1)
#corp_vocab = list(set(oov) - set(oov_rare))
#brown_tokens = [token for token in brown_nonstop if token not in oov_rare]
#brown_doc = [' '.join(brown_tokens)]

corp_vocab = list(set(oov))
brown_doc = [' '.join(brown_nonstop)]

cv = CountVectorizer(ngram_range=(1,1), vocabulary=corp_vocab)
X = cv.fit_transform(brown_doc)
Xc = (X.T * X)
Xc.setdiag(0)
coocc_ar = Xc.toarray()

mittens_model = Mittens(n=300, max_iter=1000)

new_embeddings = mittens_model.fit(
    coocc_ar,
    vocab=corp_vocab,
    initial_embedding_dict= pre_glove)

/usr/local/lib/python3.7/dist-packages/scipy/sparse/_index.py:126: SparseEfficiencyWarning: Changing the sparsity structure of a csc_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Iteration 1000: loss: 0.026411626487970352